In [5]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [6]:
pwd = os.getcwd()
filepath = os.path.join(pwd, 'dataset/titanic.csv')

In [7]:
df = pd.read_csv(filepath)
df.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,1.0,0.0,3.0,male,22.0,1.0,0.0,7.2500,S
1,1,2.0,1.0,1.0,female,38.0,1.0,0.0,71.2833,C
2,2,3.0,1.0,3.0,female,26.0,0.0,0.0,7.9250,S
3,3,4.0,1.0,1.0,female,35.0,1.0,0.0,53.1000,S
4,4,5.0,0.0,3.0,male,35.0,0.0,0.0,8.0500,S


In [8]:
def one_hot_encode(df):
    df_cat = df[["Sex", "Embarked"]]
    cat_encoder = OneHotEncoder(sparse=False)
    df_cat_1hot = cat_encoder.fit_transform(df_cat)
    feature_names = cat_encoder.get_feature_names_out(["Sex", "Embarked"])
    new_df = pd.DataFrame(df_cat_1hot, columns=feature_names, index=df.index)
    df.drop(['Sex', 'Embarked'], axis=1, inplace=True)
    df = pd.concat([df, new_df], axis=1)
    return df